# COVID-19 Selbsttest & Bayes:

In [1]:
# auto-print assignment values
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

Gegeben Coronavirus (2019-nCoV)-Antigentest Test Confusion Matrix:
* $T$: positives Testergebnis
* $C$: hat wirklich Corona (genähert durch PCR Test)

|     | $C$ | $\overline{C}$ |
|-----|-----|-----|
| $T$ | 103 |  1  |
| $\overline{T}$ |   5 | 114 |

## Unter Testbedingungen also:

### Zeilenweise: gg. Test $T$ oder $\overline{T}$, wie wahrscheinlich ist dann $C$ oder $\overline{C}$?

In [2]:
P_c_t = 103 / (103+1)  # P(C|T), positive predictive value, precision
P_nc_t = 1 / (103+1)   # P(not(C)|T), false discovery rate
P_c_t, P_nc_t

(0.9903846153846154, 0.009615384615384616)

In [3]:
P_c_nt = 5 / (5+114)     # P(C|not(T)), false omission rate
P_nc_nt = 114 / (5+114)  # P(not(C)|not(T)), negative predictive value
P_c_nt, P_nc_nt

(0.04201680672268908, 0.957983193277311)

### Spaltenweise: gg. $C$ oder $\overline{C}$, wie wahrscheinlich ist dann $T$ oder $\overline{T}$?

In [4]:
P_t_c = 103 / (103+5)  # P(T|C), true positive rate, sensitivity, power, recall
P_nt_c = 5 / (103+5)   # P(not(T)|C), false negative rate
P_t_c, P_nt_c

(0.9537037037037037, 0.046296296296296294)

In [5]:
P_t_nc = 1 / (1+114)     # P(T|not(C)), false positive rate
P_nt_nc = 114 / (1+114)  # P(not(T)|not(C)), true negative rate, specificity
P_t_nc, P_nt_nc

(0.008695652173913044, 0.991304347826087)

### Erster Eindruck:

Guter Test...

... aber der "Prior", "Prävalenz":

In [6]:
P_c = (103 + 5) / (103 + 5 + 1 + 114)  # P(C) unter Testbedingungen!

0.484304932735426

In [7]:
P_t = (103 + 1) / (103 + 5 + 1 + 114)  # P(T) unter Testbedingungen!

0.4663677130044843

## Unter realeren Bedingungen:

In [8]:
P_C = 53/100000  # aktuelle 7-Tages-Inzidenz (RKI)

0.00053

Was bedeutet das für unsere Confusion-Matrix?

Anschaulich wird aus

|     | $C$ | $\overline{C}$ |
|-----|-----|-----|
| $T$ | 103 |  1  |
| $\overline{T}$ |   5 | 114 |

durch "werfen" von $53$ erwarteten kranken Personen in die Spalce $C$ und $99,947$ erwarteten gesunden Personen in Spalte $\overline{C}$ ca. diese Tabelle mit erwarteten Personenzahlen:

|     | $C'$ | $\overline{C'}$ |
|-----|-----|-----|
| $T'$ | 50.546 |  869.104  |
| $\overline{T'}$ |   2.454 | 99,077.896 |

Wir sehen, dass die erste Zeile von den falsch positiven dominiert wird, was einfach an der viel größeren Zahl an Personen in der Spalte liegt. An der Tabelle können wir nun ablesen, dass $P(C|T) \approx \frac{50.546}{50.546+869.104} = 0.054962$, aber sie ändert sich jede Woche!

Daher lieber direkt durch Anwendung des Satzes von Bayes:

$$P(C|T) = \frac{P(T|C) P(C)}{P(T)} = \frac{P(T|C) P(C)}{P(T|C) P(C) + P(T|\overline{C}) P(\overline{C})}$$

$$P(C|\overline{T}) = \frac{P(\overline{T}|C) P(C)}{P(\overline{T})} = \frac{P(\overline{T}|C) P(C)}{P(\overline{T}|C) P(C) + P(\overline{T}|\overline{C}) P(\overline{C})}$$

Was sind also die Wahrscheinlichkeiten
- $P(C|T)$, also dass man an Covid erkrankt ist, falls ein Selbsttest positiv ist?
- $P(C|\overline{T})$, also dass man an Covid erkrankt ist, falls ein Selbsttest negativ ist?

Berechnen wir zunächst kurz $P(T)$ und $P(\overline{T})$:

In [9]:
P_T = P_t_c * P_C + P_t_nc * (1-P_C)
# P_nT = P_nt_c * P_C + P_nt_nc * (1-P_C)
P_nT = 1 - P_T
P_T, P_nT

(0.009196506441223832, 0.9908034935587762)

Auch hier sieht man wieder, dass wir uns stark vom $P(T) \approx 0.466$ unter den ursprünglichen Testbedingungen entfernt haben.

### Mein Selbsttest war positiv, bin ich krank? $P(C|T)$

In [10]:
P_C_T = (P_t_c * P_C) / P_T

0.054962497573774124

Die Wahrscheinlichkeit hierfür ist also tatsächlich eher gering, aber doch deutlich höher als der angesetzte Prior der 7-Tages-Inzidenz.

In [11]:
# zur Erinnerung:
P_C

0.00053

In [12]:
# zur Erinnerung: P(C|T) unter Testbedingungen
103/104

0.9903846153846154

Man hat also anders als unter Testbedingungen aufgrund des eher geringen realen Priors momentan nur eine geringe Wahrscheinlichkeit, dass man wirklich krank ist. (Natürlich vorausgesetzt, dass man sich nicht aufgrund eines Verdachts getestet hat.)

Daher auch die übliche Empfehlung: via PCR Test kontrollieren lassen!

Dennoch natürlich: An die geltenden Regeln halten, erstmal in Selbstquarantäne, denn $P(C|T) >> P(C)$.

### Mein Selbsttest war negativ, bin ich evtl. dennoch krank? $P(C|\overline{T})$

In [13]:
%precision %.6f
P_c_nT = (P_nt_c * P_C) / P_nT

0.000025

Das ist also durch den negativen Testausgang nochmal unwahrscheinlicher geworden als die 7-Tages-Inzidenz, also $P(C) >> P(C|\overline{T})$.

## Fazit:

Nutzen Sie die Testangebote, aber bewahren Sie die Ruhe bei einem ersten evtl. falsch positiven Testergebnis.

## Weiterführende Links:

- [Confusion Matrix](https://en.wikipedia.org/wiki/Confusion_matrix)
- [Satz von Bayes](https://de.wikipedia.org/wiki/Satz_von_Bayes), [Bayes' theorem](https://en.wikipedia.org/wiki/Bayes%27_theorem)
- [RKI Lage-/Situationsberichte](https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Situationsberichte/Gesamt.html)
- [RKI Testzahlen](https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Testzahl.html)